In [1]:
import psycopg2
import configparser
import pandas as pd
import numpy as np

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

In [3]:
SQL = """
DROP FUNCTION IF EXISTS api_get_city_stats;

CREATE OR REPLACE FUNCTION api_get_city_stats(    
	in_cityid integer,
	in_poi_category varchar,
	in_timeofday varchar,
	in_categorytype text,
	in_remove_hex_ids character[] default array[]::character[],
    in_add_hex_ids character[] default array[]::character[]	
)
    RETURNS TABLE
            (                
                groupn    	text,                                
                metric        float
            )
    LANGUAGE plpgsql
AS
$getcitystats$
BEGIN
    RETURN QUERY
	
	--- GET base statistics on H3 level
	WITH stats as (
		SELECT 
		a.h3id, 
		a.groupname,
		a.accessibility				
		FROM accessibility_stats a 
		WHERE 			
			a.cityid = in_cityid
			AND a.categorytype = in_categorytype
			AND a.poi_category = in_poi_category
			AND a.timeofday = in_timeofday
	),
	
	--- GET adjustments on H3 level
	adjustments as (
		SELECT adj.h3id, adj.groupname, adj.adjustment as accessibility
        FROM api_add_remove_catchments(
			in_remove_hex_ids, 
			in_add_hex_ids,
			in_poi_category,
	        in_timeofday,
	       	in_categorytype
		) as adj
	),
	
	-- combine base data and adjustments
	total as (
		SELECT tt.h3id, tt.groupname, SUM(tt.accessibility) as accessibility FROM (
			SELECT s.h3id, s.groupname, s.accessibility				 
				FROM stats s
			UNION 
			SELECT adjustments.h3id, adjustments.groupname, adjustments.accessibility 
				FROM adjustments
		) tt GROUP BY tt.h3id, tt.groupname
	)
	
	SELECT 
		h.groupname,           
		SUM(t.accessibility * h.population) / SUM(h.population) as metric
	FROM total as t
	JOIN h3demographics as h
	ON 	h.h3id = t.h3id AND h.groupname = t.groupname
	WHERE h.categorytype = in_categorytype
	GROUP BY h.groupname  
	ORDER BY metric DESC;
END;
$getcitystats$;

--- SELECT * FROM api_get_city_stats(1, 'Restaurants', 'morning', 'Income');

"""

execute_sql(SQL)

### Basic test if it works

In [13]:
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute(""" SELECT * FROM api_get_city_stats(%s, %s, %s, %s)""", (1, 'Clinics and Hospitals', 'morning', 'Race'))
        res = cur.fetchall()
        
pd.DataFrame(res, columns=['group', 'metric'])

,group,metric
0,Some other race,1.977093
1,Native Hawaiian and Other Pacific Islander,1.799195
2,American Indian and Alaska Native,1.735990
3,Asian,1.638350
4,White,1.479325
5,Two or more races,1.359422
6,Black or African American,1.155636


## Test if removing all POIs works

In [11]:
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""SELECT h3id FROM api_get_pois_for_city(%s, %s)""", (1, 'Clinics and Hospitals'))        
        h3ids = [r[0] for r in cur.fetchall()]
        cur.execute(""" SELECT * FROM api_get_city_stats(%s, %s, %s, %s, %s)""", (1, 'Clinics and Hospitals', 'morning', 'Race', h3ids))
        res = cur.fetchall()


df = pd.DataFrame(res, columns=['group', 'metric'])
assert np.all(np.isclose(df.metric.values, 0)), "Not all values are close to zero - please check!"
print("Test passed!")

Test passed!


### Test if adding a duplicate catchment area for every POI works

Note that the accessibility stats do not double as we are restricting to 1 POI addition in the hexagon right now..

In [34]:
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute(""" SELECT * FROM api_get_city_stats(%s, %s, %s, %s)""", (1, 'Clinics and Hospitals', 'morning', 'Race'))
        res = cur.fetchall()
        base_df = pd.DataFrame(res, columns=['group', 'metric'])

        cur.execute(""" SELECT * FROM api_get_city_stats(%s, %s, %s, %s, %s, array(
            SELECT DISTINCT pois.h3id FROM
            pois JOIN
            cityh3map on pois.h3id = cityh3map.h3id  
            WHERE category = %s AND cityh3map.cityid = %s ))""", (1, 'Clinics and Hospitals', 'morning', 'Race', [''], 'Clinics and Hospitals', 1))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['group', 'metric'])
comb = pd.merge(base_df, df, on=["group"])

assert np.all(comb.metric_x.values < comb.metric_y.values), "Accessibility index did not increase everywhere!"
print("Test passed")
comb

Test passed


,group,metric_x,metric_y
0,Some other race,1.977093,2.248519
1,Native Hawaiian and Other Pacific Islander,1.799195,2.276649
2,American Indian and Alaska Native,1.735990,2.276542
3,Asian,1.638350,2.224372
4,White,1.479325,2.048078
5,Two or more races,1.359422,1.843872
6,Black or African American,1.155636,1.421427
